# **Model Full**

In [2]:
# ==========================================
# 0. 安装所有缺失的库
# 必须先安装才能导入
# ==========================================
print("--- 正在安装 XGBoost, CatBoost 和 LightGBM 库，这可能需要一些时间 ---")
!pip install xgboost catboost lightgbm --quiet

import pandas as pd
import numpy as np
import os
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from google.colab import drive
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
# 导入新的评估指标
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, roc_auc_score

drive.mount('/content/drive')

BASE_PATH = '/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/data/processed'

# ⚠️ 初始化用于存储结果的字典
results = {}
results_full = {}
results_full_models = {}

try:
    # 加载 SMOTE后的训练集
    X_train = pd.read_csv(os.path.join(BASE_PATH, 'X_train_smote.csv'))
    y_train = pd.read_csv(os.path.join(BASE_PATH, 'y_train_smote.csv')).squeeze()
    # 加载验证集
    X_val = pd.read_csv(os.path.join(BASE_PATH, 'X_val.csv'))
    y_val = pd.read_csv(os.path.join(BASE_PATH, 'y_val.csv')).squeeze()

    # 假设 X_train_full 和 X_val_full 是 X_train/X_val 的完整副本
    X_train_full = X_train.copy()
    X_val_full = X_val.copy()

    print("✅ 所有数据集加载成功。")
except FileNotFoundError as e:
    print(f"🚨 严重错误：未找到数据文件。请确保路径和文件名正确。错误: {e}")
    raise

# ==========================================
# 辅助函数：计算所有指标
# ==========================================
def calculate_metrics(model, X_val, y_val, y_pred, model_type='tree'):
    """计算 F1, Recall, Precision, Accuracy, AUC"""
    f1 = f1_score(y_val, y_pred)
    acc = accuracy_score(y_val, y_pred)
    prec = precision_score(y_val, y_pred)
    report = classification_report(y_val, y_pred, output_dict=True)
    recall = report['1']['recall']

    # 对于能输出概率的模型，计算 AUC
    auc = np.nan
    try:
        if model_type in ['tree', 'nn', 'lgbm', 'cb', 'xgb']:
            y_prob = model.predict_proba(X_val)[:, 1]
        elif model_type == 'lsvc':
             # LSVC 需要特殊处理来获取概率，这里暂时跳过，避免引入 CalibratedClassifierCV 复杂性
             raise NotImplementedError("LSVC 暂不计算 AUC 以避免复杂性")
        else: # KNN, LR 等
            y_prob = model.predict_proba(X_val)[:, 1]

        auc = roc_auc_score(y_val, y_prob)
    except Exception as e:
        pass # 无法计算 AUC 的模型 (如 LinearSVC)

    return {
        'F1-Score (Val)': f1,
        'Recall (Val)': recall,
        'Precision (Val)': prec,
        'Accuracy (Val)': acc,
        'AUC (Val)': auc
    }


# ==========================================
# 1. 重新计算基线结果 (Model Behv. - RF)
# ==========================================
print("\n--- 1. 重新计算基线模型 (RF Behv.) ---")
BEHAVIORAL_FEATURES = [
    'StudyHours', 'DietHabit_Healthy', 'DietHabit_Moderate', 'DietHabit_Others', 'DietHabit_Unhealthy',
    'SleepDur_5-6 hours', 'SleepDur_7-8 hours', 'SleepDur_Less than 5 hours',
    'SleepDur_More than 8 hours', 'SleepDur_Others'
]
X_train_behav = X_train[BEHAVIORAL_FEATURES]
X_val_behav = X_val[BEHAVIORAL_FEATURES]

# 训练基线 RF (用于对比)
rf_model_behav = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=None, n_jobs=-1)
rf_model_behav.fit(X_train_behav, y_train)
y_val_pred_rf_behav = rf_model_behav.predict(X_val_behav)
results['RF'] = calculate_metrics(rf_model_behav, X_val_behav, y_val, y_val_pred_rf_behav, model_type='tree')
print(f"✅ 基线模型 (RF Behv.) F1-Score: {results['RF']['F1-Score (Val)']:.4f}, AUC: {results['RF']['AUC (Val)']:.4f}")

# ==========================================
# 2. 训练与评估所有全特征模型 (Model Full)
# ==========================================

# --- 2.1 随机森林 (RF Full) ---
print("\n--- 2.1 训练 RF Full ---")
rf_full_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=None, n_jobs=-1)
rf_full_model.fit(X_train_full, y_train)
y_val_pred_rf_full = rf_full_model.predict(X_val_full)
results_full['RF_Full'] = calculate_metrics(rf_full_model, X_val_full, y_val, y_val_pred_rf_full, model_type='tree')
results_full_models['RF_Full'] = rf_full_model

# --- 2.2 XGBoost (XGB Full) ---
print("--- 2.2 训练 XGB Full ---")
xgb_full_model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', n_estimators=100, random_state=42, n_jobs=-1)
xgb_full_model.fit(X_train_full, y_train)
y_val_pred_xgb_full = xgb_full_model.predict(X_val_full)
results_full['XGB_Full'] = calculate_metrics(xgb_full_model, X_val_full, y_val, y_val_pred_xgb_full, model_type='xgb')
results_full_models['XGB_Full'] = xgb_full_model

# --- 2.3 K-近邻 (KNN Full) ---
print("--- 2.3 训练 KNN Full ---")
knn_full_model = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
knn_full_model.fit(X_train_full, y_train)
y_val_pred_knn_full = knn_full_model.predict(X_val_full)
results_full['KNN_Full'] = calculate_metrics(knn_full_model, X_val_full, y_val, y_val_pred_knn_full, model_type='other')
results_full_models['KNN_Full'] = knn_full_model

# --- 2.4 线性支持向量机 (LinearSVC Full) ---
print("--- 2.4 训练 LinearSVC Full ---")
lsvc_full_model = LinearSVC(random_state=42, dual=True, max_iter=5000, class_weight=None)
lsvc_full_model.fit(X_train_full, y_train)
y_val_pred_lsvc_full = lsvc_full_model.predict(X_val_full)
results_full['LSVC_Full'] = calculate_metrics(lsvc_full_model, X_val_full, y_val, y_val_pred_lsvc_full, model_type='lsvc')
results_full_models['LSVC_Full'] = lsvc_full_model

# --- 2.5 LightGBM (LGBM Full) ---
print("--- 2.5 训练 LGBM Full ---")
lgbm_full_model = lgb.LGBMClassifier(objective='binary', metric='binary_logloss', n_estimators=100, random_state=42, n_jobs=-1, verbose=-1)
lgbm_full_model.fit(X_train_full, y_train)
y_val_pred_lgbm_full = lgbm_full_model.predict(X_val_full)
results_full['LGBM_Full'] = calculate_metrics(lgbm_full_model, X_val_full, y_val, y_val_pred_lgbm_full, model_type='lgbm')
results_full_models['LGBM_Full'] = lgbm_full_model

# --- 2.6 CatBoost (CB Full) ---
print("--- 2.6 训练 CatBoost Full ---")
cb_full_model = CatBoostClassifier(iterations=500, loss_function='Logloss', eval_metric='F1', random_seed=42, verbose=0, thread_count=-1)
cb_full_model.fit(X_train_full, y_train)
y_val_pred_cb_full = cb_full_model.predict(X_val_full)
results_full['CB_Full'] = calculate_metrics(cb_full_model, X_val_full, y_val, y_val_pred_cb_full, model_type='cb')
results_full_models['CB_Full'] = cb_full_model

print("✅ 所有 Model Full 算法训练完成。")

# ==========================================
# 3. 结果总结与对比
# ==========================================
df_results_full = pd.DataFrame(results_full).T
df_results_baseline = pd.DataFrame({'RF_Behav': results['RF']}).T
df_comparison = pd.concat([df_results_full, df_results_baseline]).sort_values(by='F1-Score (Val)', ascending=False)

print("\n============================================================")
print("     所有全特征模型性能对比 (Model Full) - 扩展指标           ")
print("============================================================")
print(df_comparison.round(4)) # 四位小数显示
print("\nAUC (Val) 对于 LSVC_Full 未计算 (该模型不直接输出概率)。")

# 确定最终教师模型 (用于知识蒸馏)
teacher_model_name = df_comparison.index[0]
if 'LSVC' in teacher_model_name:
    # 寻找树模型中的最高分作为教师 (可输出概率)
    tree_models = {k: v for k, v in results_full.items() if 'RF' in k or 'XGB' in k or 'LGBM' in k or 'CB' in k}
    best_tree_name = pd.DataFrame(tree_models).T['F1-Score (Val)'].idxmax()
    teacher_model = results_full_models[best_tree_name]
    print(f"\n⚠️ 理论最高分由 LSVC 获得，但选择 {best_tree_name} (F1: {df_results_full.loc[best_tree_name, 'F1-Score (Val)']:.4f}) 作为教师模型（可输出概率）。")
else:
    teacher_model = results_full_models[teacher_model_name]
    print(f"\n🎉 有可能的教师模型 (Teacher Model) 为: {teacher_model_name}")

print(f"性能上限 (F1-Score): {df_comparison.iloc[0]['F1-Score (Val)']:.4f}")
print("\n--- 流程已完成 ---")


--- 正在安装 XGBoost, CatBoost 和 LightGBM 库，这可能需要一些时间 ---
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 所有数据集加载成功。

--- 1. 重新计算基线模型 (RF Behv.) ---
✅ 基线模型 (RF Behv.) F1-Score: 0.7167, AUC: 0.6782

--- 2.1 训练 RF Full ---
--- 2.2 训练 XGB Full ---
--- 2.3 训练 KNN Full ---
--- 2.4 训练 LinearSVC Full ---
--- 2.5 训练 LGBM Full ---
--- 2.6 训练 CatBoost Full ---
✅ 所有 Model Full 算法训练完成。

     所有全特征模型性能对比 (Model Full) - 扩展指标           
           F1-Score (Val)  Recall (Val)  Precision (Val)  Accuracy (Val)  \
LSVC_Full          0.8797        0.8865           0.8730          0.8580   
CB_Full            0.8740        0.8808           0.8674          0.8513   
LGBM_Full          0.8729        0.8792           0.8668          0.8501   
XGB_Full           0.8684        0.8743           0.8627          0.8448   
RF_Full            0.8671        0.8694           0.8648          0.8439   
KNN_Full           0.8262        0.7943

In [3]:
import random
import os
import numpy as np
import torch
import tensorflow as tf  # 如果你用的是TensorFlow/Keras

def seed_everything(seed=42):
    """
    固定所有可能的随机种子，确保实验可复现。
    :param seed: 随机种子数值，通常常用 42, 0, 2023 等
    """
    # 1. Python 原生随机种子
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

    # 2. Numpy 随机种子 (数据处理常用)
    np.random.seed(seed)

    # 3. PyTorch 随机种子 (如果你在使用 PyTorch)
    try:
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # 如果使用多GPU

        # 这一步非常重要：强制 CuDNN 使用确定性算法
        # 虽然这会稍微降低训练速度，但能保证结果一致
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    except NameError:
        pass # 如果没装Torch则跳过

    # 4. TensorFlow 随机种子 (如果你在使用 TF/Keras)
    try:
        tf.random.set_seed(seed)
    except NameError:
        pass # 如果没装TF则跳过

    print(f"Global seed set to {seed}")

# --- 使用方法 ---
# 在你开始做任何事情之前（定义模型、切分数据前），先调用它：
seed_everything(42)

Global seed set to 42


In [82]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import f1_score, classification_report, accuracy_score, precision_score, roc_auc_score
import numpy as np
import pandas as pd
import os

# --- 路径常量 ---
BASE_PATH = '/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/data/processed'
MODEL_SAVE_PATH = os.path.join(os.path.dirname(BASE_PATH), 'models', 'nn_teacher_full.keras')

# 确保 X_train_full, X_val_full, y_train, y_val 存在 (假设它们已被加载)
if 'X_train_full' not in locals() or X_train_full.empty:
    print("🚨 致命错误：X_train_full 或 X_val_full 不存在。请运行前面的数据加载代码。")
    exit()

# ==========================================
# 0. 辅助函数：计算所有 NN 指标
# ==========================================
def calculate_nn_metrics(y_true_1d, y_prob):
    """计算 NN 模型的 F1, Recall, Precision, Accuracy, AUC"""

    # 默认阈值 0.5 预测
    y_pred = (y_prob > 0.5).astype(int)

    f1 = f1_score(y_true_1d, y_pred)
    acc = accuracy_score(y_true_1d, y_pred)
    prec = precision_score(y_true_1d, y_pred)
    auc = roc_auc_score(y_true_1d, y_prob)

    # 获取 Recall
    report = classification_report(y_true_1d, y_pred, output_dict=True)
    recall = report['1']['recall']

    return {
        'F1-Score (Val)': f1,
        'Recall (Val)': recall,
        'Precision (Val)': prec,
        'Accuracy (Val)': acc,
        'AUC (Val)': auc
    }


# ==========================================
# 1. 关键修复：扩展标签维度
# ==========================================
# 这一步是为了满足 Keras F1Score 指标的要求，将 y_train 和 y_val 转换为 (N, 1) 形状

# 转换为 numpy 数组
y_train_np = np.array(y_train)
y_val_np = np.array(y_val)

# 扩展维度
y_train_2d = np.expand_dims(y_train_np, axis=1) # 形状变为 (N, 1)
y_val_2d = np.expand_dims(y_val_np, axis=1)   # 形状变为 (N, 1)

print(f"--- 1. 标签形状修复：y_train_2d 形状: {y_train_2d.shape} ---")

# ==========================================
# 2. 训练 NN 教师模型 (全特征)
# ==========================================

print("\n--- 2. 训练 NN 教师模型 (Full Features) ---")

# 1. 模型定义
def create_teacher_nn_model(input_dim):
    # 使用 tanh 激活函数，通常在深度NN中能帮助学习更复杂的非线性特征
    model = Sequential([
        Dense(16, activation='tanh', input_shape=(input_dim,), name='dense_16'),
        Dropout(0.3),
        Dense(16, activation='tanh', input_shape=(input_dim,), name='dense_81'),
        Dropout(0.3),
        Dense(16, activation='tanh', input_shape=(input_dim,), name='dense_82'),
        Dropout(0.3),
        Dense(1, activation='sigmoid', name='output')
    ], name='NN_Teacher_Full')

    model.compile(optimizer=Adam(learning_rate=0.0005),
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.F1Score(name='f1_score', average='micro', threshold=0.5)])
    return model

# 2. 实例化模型
input_dim = X_train_full.shape[1]
nn_teacher_model = create_teacher_nn_model(input_dim)
nn_teacher_model.summary() #

# 3. 设置回调函数
early_stopping = EarlyStopping(monitor='val_f1_score', patience=10, mode='max', restore_best_weights=True)
model_checkpoint = ModelCheckpoint(
    MODEL_SAVE_PATH,
    monitor='val_f1_score',
    mode='max',
    save_best_only=True,
    verbose=0
)

# 4. 训练模型 (使用 2D 标签)
EPOCHS = 100
BATCH_SIZE = 64

print(f"\n开始训练 NN 教师模型 (Epochs: Max {EPOCHS}, Batch: {BATCH_SIZE})...")

history_nn_teacher = nn_teacher_model.fit(
    np.array(X_train_full),
    y_train_2d,
    validation_data=(np.array(X_val_full), y_val_2d),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

# 5. 加载并评估最佳模型
best_nn_teacher_model = tf.keras.models.load_model(MODEL_SAVE_PATH)

# 获取概率 (预测软标签)
y_val_prob_nn_teacher = best_nn_teacher_model.predict(np.array(X_val_full)).flatten()

# 6. 结果总结和对比 (使用辅助函数计算全部指标)
metrics_nn_teacher = calculate_nn_metrics(y_val_np, y_val_prob_nn_teacher)

results_nn_teacher = {
    'NN_Teacher_Full': metrics_nn_teacher
}
df_results_nn_teacher = pd.DataFrame(results_nn_teacher).T.round(4) # 四位小数显示

# 将 NN 教师模型的结果加入到您的总对比表格中
if 'df_comparison' in locals():
    # 确保合并时类型匹配
    df_comparison_temp = df_comparison.copy()

    # ⚠️ 确保 df_comparison 中的 NaN 值不会导致 merge 失败，并只取我们需要的列
    df_results_nn_teacher = df_results_nn_teacher[[
        'F1-Score (Val)', 'Recall (Val)', 'Precision (Val)', 'Accuracy (Val)', 'AUC (Val)'
    ]]

    df_comparison_final = pd.concat([df_comparison_temp, df_results_nn_teacher]).sort_values(by='F1-Score (Val)', ascending=False)
else:
    df_comparison_final = df_results_nn_teacher

print("\n============================================================")
print("     NN 教师模型性能总结 (Model Full) - 扩展指标           ")
print("============================================================")
print(df_comparison_final.round(4))

print(f"\n🎉 NN 教师模型 (Full) F1-Score: {df_comparison_final.loc['NN_Teacher_Full', 'F1-Score (Val)']:.4f}")
print(f"NN 教师模型已训练并保存到: {MODEL_SAVE_PATH}")

--- 1. 标签形状修复：y_train_2d 形状: (22858, 1) ---

--- 2. 训练 NN 教师模型 (Full Features) ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "NN_Teacher_Full"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                │ (None, 16)             │           752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_195 (Dropout)           │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_81 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_196 (Dropout)           │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_82 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_197 (Dropout)           │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313 (5.13 KB)

 Trainable params: 1,313 (5.13 KB)

 Non-trainable params: 0 (0.00 B)


开始训练 NN 教师模型 (Epochs: Max 100, Batch: 64)...
Epoch 1/100
358/358 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6805 - f1_score: 0.6895 - loss: 0.5964 - val_accuracy: 0.8448 - val_f1_score: 0.8659 - val_loss: 0.3564
Epoch 2/100
358/358 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8210 - f1_score: 0.8217 - loss: 0.4139 - val_accuracy: 0.8482 - val_f1_score: 0.8682 - val_loss: 0.3500
Epoch 3/100
358/358 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8303 - f1_score: 0.8302 - loss: 0.3977 - val_accuracy: 0.8515 - val_f1_score: 0.8719 - val_loss: 0.3461
Epoch 4/100
358/358 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8329 - f1_score: 0.8335 - loss: 0.3855 - val_accuracy: 0.8530 - val_f1_score: 0.8733 - val_loss: 0.3459
Epoch 5/100
358/358 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8431 - f1_score: 0.8435 - loss: 0.3769 - val_accuracy: 0.8546 - val_f1_score: 0.8752 - val_loss: 0.3445
Epoch 6/100
358/358 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8386 - f1_score: 0.8396 - loss: 

In [83]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
import os
import random

# --- 1. 确保可复现性设置 (重要!) ---
# 这对于 CV 尤为重要，确保模型初始化在每个 Fold 中是相同的
SEED_VALUE = 42
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
tf.random.set_seed(SEED_VALUE)
# ⚠️ 注意：如果使用 GPU 且需要绝对复现，需添加：
# os.environ['TF_DETERMINISTIC_OPS'] = '1'
# os.environ['TF_CUDNN_USE_AUTOTUNE'] = '0'

# --- 路径常量 (假设 X_train_full, y_train, X_val_full, y_val 已在环境中加载) ---
BASE_PATH = '/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/data/processed'
# 为 CV 创建独立的模型保存目录
CV_MODEL_DIR = os.path.join(os.path.dirname(BASE_PATH), 'models', 'nn_teacher_cv')
os.makedirs(CV_MODEL_DIR, exist_ok=True)

# ==========================================
# 2. 教师模型配置 (我们选定的最稳定配置)
# ==========================================
BEST_NN_CONFIG = {
    'structure': [8, 8, 8], # 隐藏层神经元数
    'activation': 'tanh',
    'dropout_rate': 0.4,
    'learning_rate': 0.0005
}

# ==========================================
# 3. 数据准备和标签维度修复
# ==========================================
# 为了在 Keras 中使用 KFold，我们将训练集和验证集合并
X_full = pd.concat([X_train_full, X_val_full])
y_full = pd.concat([y_train, y_val])

X_full_np = np.array(X_full)
y_full_2d = np.expand_dims(np.array(y_full), axis=1) # 转换为 (N, 1) 形状

# ==========================================
# 4. 模型定义函数
# ==========================================
def create_teacher_nn_model(input_dim, config):
    # 每次创建新模型时，重置随机种子以确保初始化一致性
    tf.random.set_seed(SEED_VALUE)

    layers = []
    # 添加输入层
    layers.append(Dense(config['structure'][0], activation=config['activation'],
                        input_shape=(input_dim,), name='dense_input'))
    layers.append(Dropout(config['dropout_rate']))

    # 添加隐藏层
    for i, units in enumerate(config['structure'][1:]):
        layers.append(Dense(units, activation=config['activation'], name=f'dense_{i+1}'))
        layers.append(Dropout(config['dropout_rate']))

    # 输出层
    layers.append(Dense(1, activation='sigmoid', name='output'))

    model = Sequential(layers, name='NN_Teacher_CV')

    model.compile(optimizer=Adam(learning_rate=config['learning_rate']),
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.F1Score(name='f1_score', average='micro', threshold=0.5)])
    return model

# ==========================================
# 5. K-Fold 交叉验证执行
# ==========================================
N_SPLITS = 5
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED_VALUE)

cv_results = []
best_global_f1 = -1.0
best_model_path = os.path.join(CV_MODEL_DIR, 'nn_teacher_best_cv.keras')

print(f"\n--- 正在对最佳配置进行 {N_SPLITS}-Fold 交叉验证 ---")
print(f"配置: {BEST_NN_CONFIG}")

# 遍历每个 Fold
for fold, (train_index, val_index) in enumerate(kf.split(X_full_np)):
    print(f"\n📢 开始训练 Fold {fold+1}/{N_SPLITS}...")

    X_train_fold, X_val_fold = X_full_np[train_index], X_full_np[val_index]
    y_train_2d_fold, y_val_2d_fold = y_full_2d[train_index], y_full_2d[val_index]
    y_val_1d_fold = np.squeeze(y_val_2d_fold) # 用于 sklearn 评估

    # 实例化模型
    nn_teacher_model = create_teacher_nn_model(X_train_fold.shape[1], BEST_NN_CONFIG)

    # 为该 Fold 定义模型保存路径
    fold_model_path = os.path.join(CV_MODEL_DIR, f'fold_{fold+1}_f1_{{val_f1_score:.4f}}.keras')

    # 设置回调函数
    early_stopping = EarlyStopping(monitor='val_f1_score', patience=15, mode='max', restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(
        fold_model_path,
        monitor='val_f1_score',
        mode='max',
        save_best_only=True,
        verbose=0
    )

    # 训练模型
    history = nn_teacher_model.fit(
        X_train_fold,
        y_train_2d_fold,
        validation_data=(X_val_fold, y_val_2d_fold),
        epochs=150, # 增加 Epochs，让 EarlyStopping 发挥作用
        batch_size=64,
        callbacks=[early_stopping, model_checkpoint],
        verbose=0
    )

    # 加载该 Fold 的最佳权重进行评估
    best_fold_f1 = np.max(history.history['val_f1_score'])
    best_fold_loss = np.min(history.history['val_loss'])

    # 评估指标
    # 注意：我们使用 restore_best_weights=True，所以 nn_teacher_model 已经是该 Fold 的最佳模型
    y_val_prob = nn_teacher_model.predict(X_val_fold).flatten()
    y_val_pred = (y_val_prob > 0.5).astype(int)

    f1 = f1_score(y_val_1d_fold, y_val_pred)
    auc = roc_auc_score(y_val_1d_fold, y_val_prob)

    # 记录结果
    cv_results.append({
        'Fold': fold + 1,
        'F1-Score': f1,
        'AUC': auc,
        'Val_Loss': best_fold_loss
    })

    # 记录全局最佳模型
    if f1 > best_global_f1:
        best_global_f1 = f1
        nn_teacher_model.save(best_model_path)
        print(f"   ✨ 新的全局最佳模型 F1: {f1:.4f} 已保存。")

# ==========================================
# 6. 结果总结
# ==========================================
df_cv_results = pd.DataFrame(cv_results)

# 计算均值和标准差
f1_mean = df_cv_results['F1-Score'].mean()
f1_std = df_cv_results['F1-Score'].std()
auc_mean = df_cv_results['AUC'].mean()
auc_std = df_cv_results['AUC'].std()

print("\n==============================================")
print(f"🏆 NN 教师模型 (8-8-8-1) {N_SPLITS}-Fold 交叉验证总结")
print("==============================================")
print(df_cv_results.round(4).to_markdown(index=False))

print(f"\n**稳定性能均值:**")
print(f"   F1-Score (Mean ± Std): {f1_mean:.4f} ± {f1_std:.4f}")
print(f"   AUC (Mean ± Std): {auc_mean:.4f} ± {auc_std:.4f}")
print(f"\n🎉 最终教师模型已保存至: {best_model_path}")


--- 正在对最佳配置进行 5-Fold 交叉验证 ---
配置: {'structure': [8, 8, 8], 'activation': 'tanh', 'dropout_rate': 0.4, 'learning_rate': 0.0005}

📢 开始训练 Fold 1/5...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
   ✨ 新的全局最佳模型 F1: 0.8705 已保存。

📢 开始训练 Fold 2/5...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

📢 开始训练 Fold 3/5...


...

169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

📢 开始训练 Fold 4/5...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

📢 开始训练 Fold 5/5...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

🏆 NN 教师模型 (8-8-8-1) 5-Fold 交叉验证总结
|   Fold |   F1-Score |    AUC |   Val_Loss |
|-------:|-----------:|-------:|-----------:|
|      1 |     0.8705 | 0.9368 |     0.3205 |
|      2 |     0.8655 | 0.9348 |     0.3233 |
|      3 |     0.8633 | 0.9319 |     0.3343 |
|      4 |     0.8678 | 0.9377 |     0.3197 |
|      5 |     0.8685 | 0.9357 |     0.3224 |

**稳定性能均值:**
   F1-Score (Mean ± Std): 0.8671 ± 0.0028
   AUC (Mean ± Std): 0.9354 ± 0.0022

🎉 最终教师模型已保存至: /content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/data/models/nn_teacher_cv/nn_teacher_best_cv.keras


In [85]:
import tensorflow as tf
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, accuracy_score
import numpy as np

# 1. 定义模型路径 (使用您提供的路径)
MODEL_PATH = '/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/data/models/nn_teacher_cv/nn_teacher_best_cv.keras'

# 2. 确保数据准备 (假设 X_val_full 和 y_val 仍然可用)
# X_val_full: 验证集特征
# y_val: 验证集真实标签 (1D numpy array 或 list)

# 转换标签为 numpy 数组 (确保是 1D 格式用于 sklearn 评估)
y_val_np = np.array(y_val)

print(f"--- 正在加载模型：{MODEL_PATH} ---")

# 3. 加载模型
try:
    best_teacher_model = tf.keras.models.load_model(MODEL_PATH)
except Exception as e:
    print(f"加载模型失败，请检查路径和文件：{e}")
    # 退出执行或返回

# 4. 在验证集上预测概率
y_val_prob = best_teacher_model.predict(X_val_full).flatten()

# 5. 计算二分类预测 (阈值 0.5)
y_val_pred = (y_val_prob > 0.5).astype(int)

# 6. 计算评估指标
final_f1 = f1_score(y_val_np, y_val_pred)
final_auc = roc_auc_score(y_val_np, y_val_prob)
final_precision = precision_score(y_val_np, y_val_pred)
final_recall = recall_score(y_val_np, y_val_pred)
final_accuracy = accuracy_score(y_val_np, y_val_pred)

print("\n==============================================")
print("🏆 教师模型 (Fold 1) 在原始 Validation Set 上的最终评估")
print("==============================================")
print(f"F1-Score (Val):     {final_f1:.4f}")
print(f"AUC (Val):          {final_auc:.4f}")
print(f"Recall (Val):       {final_recall:.4f}")
print(f"Precision (Val):    {final_precision:.4f}")
print(f"Accuracy (Val):     {final_accuracy:.4f}")
print("==============================================")

# 7. 准备知识蒸馏所需的软标签 (Soft Targets)
# 软标签是教师模型最重要的知识
y_soft_targets = best_teacher_model.predict(X_train_full).flatten()
y_val_soft_targets = y_val_prob # 验证集软标签就是刚才计算的概率

print("\n--- 知识蒸馏数据已生成 ---")
# 假设 X_train_full 和 X_val_full 已经加载

# --- 流程进入知识蒸馏阶段 ---

--- 正在加载模型：/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/data/models/nn_teacher_cv/nn_teacher_best_cv.keras ---
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step

🏆 教师模型 (Fold 1) 在原始 Validation Set 上的最终评估
F1-Score (Val):     0.8792
AUC (Val):          0.9257
Recall (Val):       0.8824
Precision (Val):    0.8760
Accuracy (Val):     0.8580
715/715 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step

--- 知识蒸馏数据已生成 ---
